# Simple Tidal and Battery Hybrid Plant Example
---
In this example, we will walk through the process of simulating a hybrid renewable energy system including both tidal energy and battery energy storage.

### Import Required Modules
We start by importing the necessary modules and setting up our working environment.

In [16]:
from hopp.simulation import HoppInterface

### Create the HOPP Model
To generate the HOPP Model, instantiate the `HoppInterface` class and supply the required YAML configuration with the technology configuration and site information.

For the site information, the tidal resource data **must be pre-loaded** in the format given in the `Tidal_resource_timeseries.csv`.

The tidal technology configuration requires the device rating (kw), power curve of tidal energy device as function of stream speeds (kW), and number of devices. Additionally there's a variable called `tidal_resource`, which is required for model instantiation but doesn't impact a timeseries simulation.

Note that the tidal model doesn't come with a default financial model. To address this, you must establish the `CustomFinancialModel` from HOPP.

The `default_fin_config` contains all of the necessary parameters for the financial calculations.

To maintain consistency across technologies, apply the `default_fin_config` to each technology configuration. This ensures that all technologies use the same financial model throughout the HOPP Model.

In [17]:
hi = HoppInterface("./inputs/10-tidal-battery.yaml")

### Add Tidal Cost Model Inputs
Add the inputs to run the tidal technology cost model and instantiate `MHKCosts`.

Tidal costs are available for the Reference Model 1: Tidal Current Turbine. More information about the reference models and their associated costs can be found in the [Reference Model Project](https://energy.sandia.gov/programs/renewable-energy/water-power/projects/reference-model-project-rmp/).

In [18]:
cost_model_inputs = {
	'reference_model_num':1,
	'water_depth': 100,
	'distance_to_shore': 80,
	'number_rows': 2,
	'device_spacing':600,
	'row_spacing': 600,
	'cable_system_overbuild': 20
}
hi.system.tidal.create_mhk_cost_calculator(cost_model_inputs)

### Run the Simulation
Simulate the hybrid renewable energy system for a specified number of years (in this case, 25 years).

In [19]:
hi.simulate(25)

### Retrieve and Display Results
Access the simulation results, including annual energies, capacity factors and net present values (NPVs), and print them to the console.

You can see that the Annual Energy Production for "tidal" is different than "hybrid". Part of that difference is due to the battery but also the "hybrid" system is limited by the grid interconnection limit, which in this case is less than the rated capacity of the tidal farm (interconnection limit: 22000kW and tidal farm: 22300kW).

In [20]:
hybrid_plant = hi.system

aeps = hybrid_plant.annual_energies
npvs = hybrid_plant.net_present_values
cf = hybrid_plant.capacity_factors

print("Annual Energy Production")
print(aeps)
print("Net Present Value")
print(npvs)
print("Capacity Factors")
print(cf)

Annual Energy Production
{"tidal": 60625515.491999194, "battery": 62256.91807749245, "hybrid": 60446486.28573774}
Net Present Value
{"tidal": -124990852.96205442, "battery": -32620759.796329703, "hybrid": -157608702.59276733}
Capacity Factors
{"tidal": 31.034623078812785, "battery": 0, "hybrid": 16.377843736864506}
